In [ ]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os
import time
import random

# Load the API Keys
load_dotenv()

class PersistentDojo:
    def __init__(self, api_keys, verbose=True):
        self.keys = api_keys
        self.current_key_idx = 0
        self.verbose = verbose
        self.history = []  # Global Memory
        self._init_client()

    def _init_client(self):
        self.client = genai.Client(api_key=self.keys[self.current_key_idx])

    def _rotate(self, reason="Generic"):
        self.current_key_idx = (self.current_key_idx + 1) % len(self.keys)
        if self.verbose:
            print(f"\n[Key Swap] Sensei #{self.current_key_idx} was exhausted ({reason}).")
            print(f"👉 Switching to Sensei #{self.current_key_idx + 1}...")
        self._init_client()

    def chat(self, user_input, model="gemini-2.5-flash-lite"):
        attempts = 0
        while attempts < len(self.keys):
            try:
                # 1. Start session with Global Memory
                chat_session = self.client.chats.create(
                    model=model,
                    history=self.history
                )
                
                # 2. Send Message
                response = chat_session.send_message(user_input)
                
                # 3. MANUAL HISTORY UPDATE
                # Since 'chat_session.history' isn't available, we construct the
                # history entries manually using the SDK's types.
                
                # Add User Turn
                self.history.append(types.Content(
                    role="user",
                    parts=[types.Part(text=user_input)]
                ))
                
                # Add Model Turn
                self.history.append(types.Content(
                    role="model",
                    parts=[types.Part(text=response.text)]
                ))
                
                return response.text

            except Exception as e:
                err_str = str(e).upper()
                
                # CASE 1: 503 Service Unavailable - Wait and Retry the SAME key
                if "503" in err_str or "UNAVAILABLE" in err_str:
                    wait_time = (0.5 * (2 ** attempts)) + random.random()
                    if self.verbose:
                        print(f"⚠️ [503] Server overloaded. Retrying key #{self.current_key_idx + 1} in {wait_time:.2f}s...")
                    time.sleep(wait_time)
                    attempts += 1
                    continue 
                
                # CASE 2: 429 Quota Exhausted - Rotate to the NEXT key immediately
                elif "429" in err_str or "RESOURCE_EXHAUSTED" in err_str:
                    self._rotate("429")
                    break # Break inner loop to try the next key
                
                else:
                    return f"Unexpected Error: {e}"
                
        
        return "All Senseis are currently meditating. Please try again in 60 seconds."
    

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    # Retrieve keys from environment variables
    raw_keys = [
        os.getenv("DEFAULT"),
        os.getenv("HACK1"),
        os.getenv("HACK2"),
        os.getenv("HACK3")
    ]

    # Filter out None values (in case a key is missing from .env)
    valid_keys = [k for k in raw_keys if k is not None]

    if not valid_keys:
        print("Error: No valid API keys found in .env file.")
    else:
        # Initialize Dojo
        dojo = PersistentDojo(valid_keys)
        print(f"Loaded {len(valid_keys)} API keys. Session started.")
        print("Type 'quit' or 'exit' to end the session.\n")

        # Interactive Loop
        while True:
            try:
                # Visual Indicator of which key is active
                current_key_display = f"[Key #{dojo.current_key_idx + 1}]"
                
                user_text = input(f"{current_key_display} You: ")

                # Exit condition
                if user_text.lower() in ["quit", "exit"]:
                    print("Exiting Dojo...")
                    break
                
                # Skip empty inputs
                if not user_text.strip():
                    continue

                # Get response
                response = dojo.chat(user_text)
                print(f"Dojo: {response}\n")

            except KeyboardInterrupt:
                # Handle Ctrl+C gracefully
                print("\nExiting Dojo...")
                break